Basic classification of stadsarchief images
Network based on: https://towardsdatascience.com/build-your-own-convolution-neural-network-in-5-mins-4217c2cf964f

In [1]:
import numpy as np
import math
from PIL import Image
from scipy import misc
import keras.backend as K

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.initializers import RandomUniform
from keras.layers import LeakyReLU

from stats import list_stats
from data import load_data

Using TensorFlow backend.


In [2]:
# IMG_DIR = 'examples/0-src/200x200/';
# LABEL_DIR = 'examples/0-src/labels/';
IMG_DIR = 'examples/0-src/beeldbank-scraped_set/50x50/'
LABEL_DIR = 'examples/0-src/beeldbank-scraped_set/labels/'
img_dim = (50, 50, 3);

[X, Y_yaml] = load_data(IMG_DIR, LABEL_DIR)
Y_type = np.array([y.get('type') for y in Y_yaml])

def shuffle(X, Y):
    indices = np.random.permutation(X.shape[0])
    X_shuffled = X[indices]
    Y_shuffled = Y[indices]
    return (X_shuffled, Y_shuffled)
# print(X.shape)

print('shuffling X and Y')
X_shuffled, Y_shuffled = shuffle(X, Y_type)
X = X_shuffled
Y_type = Y_shuffled

# def simplify_Y(Y):
#     return np.array([y if y == 'blueprint' else 'other' for y in Y])

# Y_type = simplify_Y(Y_type)

first few ids: ['000000091010', '000000091053', '000000091089', '000000091123', '000000091242']
shuffling X and Y


In [3]:
print(f"shape X: {X.shape}")
print(f"shape Y: {Y_type.shape}")

list_stats(Y_type)

shape X: (1201, 50, 50, 3)
shape Y: (1201,)
classes: 5
> 5 count classes: [['bouwtekening', 338], ['foto', 250], ['kaart', 30], ['prent', 575], ['affiche', 8]]
largest class: prent, count: 575
total count: 1201
score to beat: 0.4787676935886761


In [4]:
# Convert string to idx
types = list(set(Y_type))
num_classes = len(types)
print(f"number of classes: {num_classes}")
print(f"types encountered: {types}")

def convert_to_index(Y, types):
    return np.array([types.index(y) for y in Y])

Yind = convert_to_index(Y_type, types)
print(Y_type[:3])
print(Yind[:3])

list_stats(Yind)

number of classes: 5
types encountered: ['bouwtekening', 'foto', 'kaart', 'prent', 'affiche']
['prent' 'prent' 'foto']
[3 3 1]
classes: 5
> 5 count classes: [[0, 338], [1, 250], [2, 30], [3, 575], [4, 8]]
largest class: 3, count: 575
total count: 1201
score to beat: 0.4787676935886761


In [5]:
# def set_class(Y, class_idx):
#     print(Y)
#     Y[:] = class_idx
#     print(Y)
#     return Y
# # YInd = set_class(Yind, 0)

In [6]:
Y = keras.utils.to_categorical(Yind, num_classes)  # convert class vectors to binary class matrices
print(Yind.shape)
print(Y.shape)

(1201,)
(1201, 5)


In [7]:
# Split to train and test
def split_data(X, Y, size):
    X_train = X[:N]
    Y_train = Y[:N]
    X_test = X[N:]
    Y_test = Y[N:]
    return (X_train, Y_train), (X_test, Y_test)

total = X.shape[0]
N = math.floor(0.8 * total)
(X_train, Y_train), (X_test, Y_test) = split_data(X, Y, N)
print("")
print("split data shapes:")
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


split data shapes:
(960, 50, 50, 3)
(960, 5)
(241, 50, 50, 3)
(241, 5)


In [8]:
print("stats train: ")
# list_stats(Y_train)
print("stats test: ")
# list_stats(Y_test)

stats train: 
stats test: 


In [16]:
# resize = 40
# print("split data shapes:")
# print(X_train.shape)
# print(Y_train.shape)
# print(X_test.shape)
# print(Y_test.shape)
# X_train = X_train[:, :resize, :resize, :]
# X_test = X_test[:, :resize, :resize, :]
# print("split data shapes:")
# print(X_train.shape)
# print(Y_train.shape)
# print(X_test.shape)
# print(Y_test.shape)
# img_dim = (resize, resize, 3);

split data shapes:
(960, 50, 50, 3)
(960, 5)
(241, 50, 50, 3)
(241, 5)
split data shapes:
(960, 40, 40, 3)
(960, 5)
(241, 40, 40, 3)
(241, 5)


In [25]:
# def build_model(num_classes, img_dim):
#     model = Sequential()
#     model.add(Conv2D(32, kernel_size=(3, 3),
#                      activation='relu',
#                      input_shape=img_dim))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
#     model.add(Flatten())
#     model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes, activation='softmax'))
#     return model

def build_model(num_classes, img_dim):
    print(img_dim)
    model = Sequential()
    model.add(Conv2D(2, kernel_size=(3, 3),
                     activation='linear',
                     input_shape=img_dim,
                     kernel_initializer = RandomUniform()
                    ))
    model.add(LeakyReLU())
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))

    model.add(Flatten())    
#     model.add(Flatten(input_shape=img_dim))
    model.add(Dense(16, activation='linear', kernel_initializer = RandomUniform()))
    model.add(LeakyReLU())
#     model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

model = build_model(num_classes, img_dim)
model.summary()

(40, 40, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 38, 38, 2)         56        
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 38, 38, 2)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 2888)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                46224     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 85        
Total params: 46,365
Trainable params: 46,365
Non-trainable params: 0
____________________________________________________________

In [26]:
batch_size = 200
epochs = 50

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def train(model, X_train, Y_train, X_test, Y_test, batch_size, epochs):
    model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
             optimizer=keras.optimizers.Adam(
                 lr=0.0003,
# #                  lr=1e-2,
# #                  epsilon=1e-6
             ),
              metrics=['accuracy'])

    model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              
    validation_data=(X_test, Y_test))
train(model, X_train, Y_train, X_test, Y_test, batch_size, epochs)

Train on 960 samples, validate on 241 samples
Epoch 1/50
960/960 [==============================] - 1s 802us/step - loss: 11.9339 - acc: 0.2208 - val_loss: 11.2486 - val_acc: 0.2988
Epoch 2/50
960/960 [==============================] - 0s 194us/step - loss: 11.5193 - acc: 0.2771 - val_loss: 10.7902 - val_acc: 0.2988
Epoch 3/50
960/960 [==============================] - 0s 192us/step - loss: 8.9130 - acc: 0.2240 - val_loss: 6.0821 - val_acc: 0.0996
Epoch 4/50
960/960 [==============================] - 0s 187us/step - loss: 4.5103 - acc: 0.2542 - val_loss: 3.0154 - val_acc: 0.3320
Epoch 5/50
960/960 [==============================] - 0s 193us/step - loss: 2.3985 - acc: 0.3406 - val_loss: 2.1774 - val_acc: 0.3942
Epoch 6/50
960/960 [==============================] - 0s 190us/step - loss: 1.9625 - acc: 0.4802 - val_loss: 1.8115 - val_acc: 0.3817
Epoch 7/50
960/960 [==============================] - 0s 193us/step - loss: 1.5708 - acc: 0.4615 - val_loss: 1.7068 - val_acc: 0.4855
Epoch 8/50
9

In [20]:
score = model.evaluate(X_train, Y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train loss: 0.04810360854802032
Train accuracy: 0.9958333333333333
Test loss: 1.019663213694244
Test accuracy: 0.8049792541013219


In [21]:
show_cnt = 30
print(f"types: {types}")

print("train predictions, truth")
print(model.predict_classes(X_train)[:show_cnt])
print(np.argmax(Y_train, axis=1)[:show_cnt])

print("test predictions, truth")
print('[' + ' '.join(str(x)[-1:] for x in range(show_cnt)) )
print(model.predict_classes(X_test)[:show_cnt])
print(np.argmax(Y_test, axis=1)[:show_cnt])

types: ['bouwtekening', 'foto', 'kaart', 'prent', 'affiche']
train predictions, truth
[3 3 1 1 1 1 0 3 3 0 3 3 3 3 3 0 3 3 3 3 3 3 3 3 0 3 3 0 3 3]
[3 3 1 1 1 1 0 3 3 0 3 3 3 3 3 0 3 3 3 3 3 3 3 3 0 3 3 0 3 3]
test predictions, truth
[0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9
[0 0 1 0 3 3 3 3 1 0 3 1 3 3 0 0 0 0 1 3 2 3 3 0 3 3 0 0 0 0]
[0 0 1 0 3 3 1 3 1 0 3 0 3 3 0 1 0 0 1 3 0 3 3 1 3 1 0 0 0 0]


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

def show_image(data, idx):
    image=X_train[idx, :, :, :]
#     print(image.shape)
#     img = Image.fromarray(image, 'RGB')
#     img.show()
    plt.axis("off")
    plt.imshow(image)
    plt.show()

In [ ]:
show_image(X_test, 11)